## Cleaning Judicial Autonomy Dataset

### 1 Load Dataset

In [1]:
import yaml

import pandas as pd
import numpy as np

In [2]:
try:
    with open('../config.yaml') as file:
        config = yaml.safe_load(file)
except:
    print("Yaml file not found.")

In [3]:
# Load dataset q1
q1_df = pd.read_csv(config['input_data']['q1_dataset'])
q1_df.head()

,username,country,country_code,question_no,uid,year,answer,value_na,value
0,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2000,Mixed system [0.5],0.5,0.5
1,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2001,Mixed system [0.5],0.5,0.5
2,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2002,Mixed system [0.5],0.5,0.5
3,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2003,Mixed system [0.5],0.5,0.5
4,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2004,Mixed system [0.5],0.5,0.5


**Output -- DataFrame:** `q1_df`

### 2 Long Version of Data Cleaning -- Step by Step

In [30]:
# Pivot long dataframe to wide dataframe
q1_pivoted = q1_df.pivot(index=['username', 'country', 'country_code', 'year'], 
                      columns=['uid'], 
                      values=['value'])

# Reset index and flattening multi-level column names
# Use lambda function to access tuple storing the multi-level column names
q1_pivoted.reset_index(inplace=True)
q1_pivoted.columns = [col[1] if col[1] else col[0] for col in q1_pivoted.columns]
#q1_pivoted

In [31]:
# Filter out countries / users
remove_usernames = ('ADMIN123', 'ALB22uFF4m','BEL22cEw8t', 'BIH22q2nOU', 'DNK22KFh1N', 'MNE22N8NJv', 'NLD22Ba53p', 'SRB22L4wbh')
q1_countries_cleaned = q1_pivoted[~q1_pivoted['username'].isin(remove_usernames)]
#q1_countries_cleaned

In [9]:
# Replace country names
q1_countries_cleaned.loc[:,'country'] = q1_countries_cleaned['country'].replace({
    "Czech Republic": "Czechia",
    "Republic of Albania": "Albania",
    "Republic of Serbia": "Serbia",
    "Bosnia and Herzegovina (BiH)": "Bosnia and Herzegovina",
    "Montenegro (MON)": "Montenegro",
    "Kingdom of Belgium": "Belgium"})
#q1_countries_cleaned

NameError: name 'q1_countries_cleaned' is not defined

In [7]:
q1_countries_cleaned.shape

(920, 62)

In [33]:
# Drop columns that contain '_subj' (i.e. subjective answers)
q1_subj_cleaned = q1_countries_cleaned.drop(list(q1_countries_cleaned.filter(regex='_subj')), axis=1)

In [34]:
# Check for missing values across columns (in percentage) 
q1_check_nan = q1_subj_cleaned.isna().mean().sort_values(ascending=False)*100
q1_check_nan.head()

q1c2_retireage    85.108696
q1c2_jubonus      51.195652
q1c3_immunlift    31.304348
q1c3_evalints     25.326087
q1c4_whochair     19.891304
dtype: float64

In [35]:
# Filter out all columns with >20% of missing values
q1_columns_cleaned = q1_subj_cleaned.drop(columns = ['q1c2_retireage', 'q1c2_jubonus', 'q1c3_immunlift', 'q1c3_evalints'])

In [36]:
q1_cleaned_manually = q1_columns_cleaned.copy()

**Output -- DataFrame:** `q1_cleaned_manually`

### 3 Short Version of Data Cleaning -- User-defined Function

In [4]:
# Function to clean raw dataset

def cleaning_judicial_autonomy_data(q1_df):
    """
    Objective: Basic data cleaning of raw Judicial Autonomy Dataset.
    Input data: raw Judicial Autonomy Dataset.
    """
    
    # Step 1: Create copy of dataframe
    df = q1_df.copy()
    
    # Step 2: Adjust dataframe shape
    # Pivot long dataframe to wide dataframe
    df_pivoted = df.pivot(index=['username', 'country', 'country_code', 'year'], 
                          columns=['uid'], 
                          values=['value'])
    
    # Reset index and flattening multi-level column names
    df_pivoted.reset_index(inplace=True)
    df_pivoted.columns = [col[1] if col[1] else col[0] for col in df_pivoted.columns]

    # Step 3: Clean country and user names
    # Remove usernames
    remove_usernames = ('ADMIN123', 
                        'ALB22uFF4m',
                        'BEL22cEw8t', 
                        'BIH22q2nOU', 
                        'DNK22KFh1N', 
                        'MNE22N8NJv', 
                        'NLD22Ba53p', 
                        'SRB22L4wbh')
    df_countries_cleaned = df_pivoted[~df_pivoted['username'].isin(remove_usernames)]

    # Replace country names
    df_countries_cleaned.loc[:,'country'] = df_countries_cleaned['country'].replace({
        'Czech Republic': 'Czechia',
        'Republic of Albania': 'Albania',
        'Republic of Serbia': 'Serbia',
        'Bosnia and Herzegovina (BiH)': 'Bosnia and Herzegovina',
        'Montenegro (MON)': 'Montenegro',
        'Kingdom of Belgium': 'Belgium'})
    
    # Step 4: Remove columns
    # Drop columns that contain '_subj'
    # Drop columns with more than 20% values missing values ['q1c2_jubonus', 'q1c2_retireage', 'q1c3_evalints', 'q1c3_immunlift']
    columns_nan_percentage = df_countries_cleaned.isna().mean()*100
    columns_nan_20_percent = columns_nan_percentage[columns_nan_percentage > 20].index
    df_cleaned = df_countries_cleaned.drop(columns=columns_nan_20_percent)

    return df_cleaned

**Output -- function:** `cleaning_judicial_autonomy_data()`

In [12]:
# Clean dataset
q1_cleaned = cleaning_judicial_autonomy_data(q1_df) 

# Export dataset
q1_cleaned.to_csv(config['output_data']['q1_dataset'], index=False)

# Display dataset
q1_cleaned

,username,country,country_code,year,q1c1_apjuac,q1c1_apjufc,q1c1_apjuhc,q1c1_appealac,q1c1_appealfc,q1c1_appealhc,...,q1c3_sanctscale,q1c4_casealloc,q1c4_competence,q1c4_manbudget,q1c4_reasondecis,q1c4_regbudget,q1c4_sameright,q1c4_whochair,q1c4_whocharge,q1c4_whoselect
46,ALB33wGG5n,Albania,ALA,2000,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
47,ALB33wGG5n,Albania,ALA,2001,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
48,ALB33wGG5n,Albania,ALA,2002,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
49,ALB33wGG5n,Albania,ALA,2003,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
50,ALB33wGG5n,Albania,ALA,2004,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,UKR22Gf5Kx,Ukraine,UKR,2018,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1100,UKR22Gf5Kx,Ukraine,UKR,2019,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1101,UKR22Gf5Kx,Ukraine,UKR,2020,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1102,UKR22Gf5Kx,Ukraine,UKR,2021,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0


**Output -- DataFrame:** `q1_cleaned`

### 4 Alternative option: recode fuzzy values as binary values

In [13]:
# Function to recode all fuzzy values (i.e. 0.33, 0.5, 0.67) as binary values (conservative coding, i.e. value <= 0.5 as 0.0)

def recoding_fuzzy_to_binary(q1_df):
    """
    Objective: Recode all fuzzy values as binary values (conservative coding).
    Input data: raw Judicial Autonomy Dataset.
    Next step: clean Judicial Autonomy Dataset.
    """
    
    df_fuzzy_values_recoded = q1_df.copy()
    float_columns = df_fuzzy_values_recoded.select_dtypes(include=float).columns

    for col in float_columns:
        df_fuzzy_values_recoded[col] = df_fuzzy_values_recoded[col].apply(lambda value: 0.0 if value <= 0.5 else 1.0)

    return df_fuzzy_values_recoded

**Output -- function:** `recoding_fuzzy_to_binary()`

In [15]:
# Recode fuzzy values
q1_fuzzy_binary_recoded = recoding_fuzzy_to_binary(q1_df)

# Clean dataset
q1_cleaned_binary = cleaning_judicial_autonomy_data(q1_fuzzy_binary_recoded) 

# Export dataset
q1_cleaned_binary.to_csv(config['output_data']['q1_dataset_binary'], index=False)

# Display dataset
q1_cleaned_binary

,username,country,country_code,year,q1c1_apjuac,q1c1_apjufc,q1c1_apjuhc,q1c1_appealac,q1c1_appealfc,q1c1_appealhc,...,q1c4_casealloc,q1c4_competence,q1c4_manbudget,q1c4_reasondecis,q1c4_regbudget,q1c4_sameright,q1c4_subj,q1c4_whochair,q1c4_whocharge,q1c4_whoselect
46,ALB33wGG5n,Albania,ALA,2000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
47,ALB33wGG5n,Albania,ALA,2001,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
48,ALB33wGG5n,Albania,ALA,2002,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
49,ALB33wGG5n,Albania,ALA,2003,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
50,ALB33wGG5n,Albania,ALA,2004,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,UKR22Gf5Kx,Ukraine,UKR,2018,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1100,UKR22Gf5Kx,Ukraine,UKR,2019,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1101,UKR22Gf5Kx,Ukraine,UKR,2020,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1102,UKR22Gf5Kx,Ukraine,UKR,2021,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


**Output -- DataFrame:** `q1_cleaned_binary`